# Анализ Географической Уязвимости Владивостока

In [18]:
import sqlite3
import pandas as pd
from datetime import datetime

# Подключение к базе данных
DB_PATH = '../databases/dataset.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

print(f"Подключение к базе данных {DB_PATH} успешно установлено.")

Подключение к базе данных ../databases/dataset.db успешно установлено.


In [19]:
# Запрос для извлечения всех отключений
query_raw_blackouts = """
SELECT
    id,
    start_date,
    end_date,
    type
FROM
    blackouts;
"""
blackouts_df = pd.read_sql_query(query_raw_blackouts, conn)

# Преобразование строк в datetime объекты
blackouts_df['start_dt'] = pd.to_datetime(blackouts_df['start_date'])
blackouts_df['end_dt'] = pd.to_datetime(blackouts_df['end_date'])

# Расчет длительности в часах
blackouts_df['duration_hours'] = (blackouts_df['end_dt'] - blackouts_df['start_dt']).dt.total_seconds() / 3600

# Ограничим анализ отключениями, длительность которых разумна (например, не более 30 дней)
blackouts_df = blackouts_df[blackouts_df['duration_hours'] > 0]
blackouts_df = blackouts_df[blackouts_df['duration_hours'] < 720] # Менее 30 дней

# Создаем временную таблицу в памяти SQLite для агрегации
blackouts_df.to_sql('temp_blackouts_processed', conn, if_exists='replace', index=False)

print(f"Обработано {len(blackouts_df)} отключений.")

Обработано 25073 отключений.


In [20]:
# Запрос для агрегации по крупным народным районам
query_big_folk = """
SELECT
    T1.name AS big_folk_district_name,
    COUNT(T2.blackout_id) AS total_blackouts_count,
    ROUND(SUM(T3.duration_hours), 2) AS total_duration_hours,
    ROUND(AVG(T3.duration_hours), 2) AS avg_duration_hours
FROM
    big_folk_districts AS T1
JOIN
    buildings AS T4 ON T1.id = T4.big_folk_district_id
JOIN
    blackouts_buildings AS T2 ON T4.id = T2.building_id
JOIN
    temp_blackouts_processed AS T3 ON T2.blackout_id = T3.id
GROUP BY
    T1.name
ORDER BY
    total_blackouts_count DESC;
"""

df_big_folk = pd.read_sql_query(query_big_folk, conn)

print("## 🥇 ТОП-10 Крупных Народных Районов по Частоте Отключений")
print(df_big_folk.head(10).to_markdown(index=False))

print("\n---")

print("## ⏱️ Крупные Народные Районы по Суммарной Длительности (часы)")
print(df_big_folk.sort_values(by='total_duration_hours', ascending=False).head(10).to_markdown(index=False))

## 🥇 ТОП-10 Крупных Народных Районов по Частоте Отключений
| big_folk_district_name   |   total_blackouts_count |   total_duration_hours |   avg_duration_hours |
|:-------------------------|------------------------:|-----------------------:|---------------------:|
| Центр                    |                   14128 |               975004   |                69.01 |
| Чуркин                   |                   13865 |               527719   |                38.06 |
| Вторая Речка             |                   13206 |               722006   |                54.67 |
| 64, 71 микрорайоны       |                   10511 |               319999   |                30.44 |
| Трудовое                 |                   10424 |                86256.6 |                 8.27 |
| Садгород                 |                    8501 |                68645.4 |                 8.07 |
| Столетие                 |                    8372 |               548876   |                65.56 |
| Борисенко   

In [21]:
# Запрос для агрегации по официальным районам
query_official_district = """
SELECT
    T1.name AS official_district_name,
    COUNT(T2.blackout_id) AS total_blackouts_count,
    ROUND(SUM(T3.duration_hours), 2) AS total_duration_hours
FROM
    districts AS T1
JOIN
    buildings AS T4 ON T1.id = T4.district_id
JOIN
    blackouts_buildings AS T2 ON T4.id = T2.building_id
JOIN
    temp_blackouts_processed AS T3 ON T2.blackout_id = T3.id
GROUP BY
    T1.name
ORDER BY
    total_blackouts_count DESC;
"""

df_official = pd.read_sql_query(query_official_district, conn)

print("## 🥈 Официальные Районы: Частота и Длительность Отключений")
print(df_official.to_markdown(index=False))

## 🥈 Официальные Районы: Частота и Длительность Отключений
| official_district_name   |   total_blackouts_count |   total_duration_hours |
|:-------------------------|------------------------:|-----------------------:|
| Советский район          |                   53451 |            1.25932e+06 |
| Ленинский район          |                   40159 |            1.82571e+06 |
| Первомайский район       |                   34427 |            1.34679e+06 |
| Первореченский район     |                   28527 |            1.57834e+06 |
| Фрунзенский район        |                   16843 |            1.08292e+06 |


In [22]:
# Запрос для агрегации по улицам
query_street = """
SELECT
    T1.name AS street_name,
    COUNT(T2.blackout_id) AS total_blackouts_count,
    ROUND(SUM(T3.duration_hours), 2) AS total_duration_hours,
    COUNT(DISTINCT T4.id) AS affected_buildings_count
FROM
    streets AS T1
JOIN
    buildings AS T4 ON T1.id = T4.street_id
JOIN
    blackouts_buildings AS T2 ON T4.id = T2.building_id
JOIN
    temp_blackouts_processed AS T3 ON T2.blackout_id = T3.id
GROUP BY
    T1.name
HAVING 
    total_blackouts_count > 100 -- Фильтр для значимых улиц
ORDER BY
    total_blackouts_count DESC
LIMIT 15;
"""

df_street = pd.read_sql_query(query_street, conn)

print("## 🥉 ТОП-15 Улиц-Хотспотов (по частоте отключений, с учетом 100+ инцидентов)")
print(df_street.to_markdown(index=False))

## 🥉 ТОП-15 Улиц-Хотспотов (по частоте отключений, с учетом 100+ инцидентов)
| street_name                  |   total_blackouts_count |   total_duration_hours |   affected_buildings_count |
|:-----------------------------|------------------------:|-----------------------:|---------------------------:|
| 100-летия Владивостока пр-кт |                    3607 |               191618   |                        317 |
| Нейбута ул.                  |                    3525 |                94091.7 |                        155 |
| Адмирала Кузнецова ул.       |                    3229 |               100535   |                         80 |
| Красного Знамени пр-кт       |                    3212 |               130969   |                        147 |
| Калинина ул.                 |                    3172 |               120168   |                        298 |
| Русская ул.                  |                    3023 |               183395   |                        291 |
| Луговая ул.      

In [23]:
# Удаление временной таблицы
cursor.execute("DROP TABLE IF EXISTS temp_blackouts_processed;")
conn.commit()
print("Временная таблица temp_blackouts_processed удалена. Датасет возвращен к исходному состоянию.")
conn.close()
print("\nБаза данных закрыта.")

Временная таблица temp_blackouts_processed удалена. Датасет возвращен к исходному состоянию.

База данных закрыта.
